# Consumer Expenditure Trends 2006-2020 
## Ben Warzel
### 5/8/2025

*Note: generative AI was used in this project to assist in data analysis*

### Introduction
The period of 2007-2010 in American Economics is referred to as the 'Great Recession'. Following a collapse of the American housing 'bubble' or ideal economic balance in the housing market, and a crisis caused by banks giving too many loans to too many people, the general household wealth and value fell dramatically.

According to the Bureau of Labor Statistics, by 2005 consumers were spending 97% of their income. During the Great Recession, that number dropped to 92% due to higher mortage debt and lower home equity. People had to try to save more of their money, and the credit available to them shrank [2010 IMF Report]. The BLS also has data on employment statistics for the same time period, which we will examine later, but essentially all areas of employment saw a decrease in the workforce after the Great Recession. 

I chose this particular set of consumer expenditure data because it covers the majority of my childhood and adolescence, and I was curious to see the impacts of events like the 2008 housing crisis on the average American's spending habits. The rise and prevalence of credit cards into the 2010s is something else that might affect how people spend, although the credit limits were not as high then as they are now.  

The main categories I chose to focus on here are Housing, Healthcare, Food, and Transportation. I also attempt to answer questions about extraneous spending; how the consumption of alcohol and tobacco changes over time for example, or the section labeled "Entertainment" which includes things like going to movies or concerts. We would assume that as the housing market struggled, people began spending less on non-essential items. Generally we can see this pattern represented in the data and charts, but there is another expenditure that rises sharply in the late 2010s and into the 2020s: Education. This is another area of spending that lends itself to a growing Savings account; as most households will be saving to send their children to college if they are not still paying off their own student loans. 

### Methods
The original datasets were obtained from the BLS website, where I was able to download documents containing detailed line-by-line breakdowns of spending by year. These data sets were made up of information collected from voluntary Interview Surveys detailing larger expenditures and Diary Surveys which kept track of smaller weekly spending. There is demographic information provided as well, in the form of "Percent Distribution" (or relative total frequency out of 100) for gender and race. I believe this is an attempt to provide further context for the data, but the demographics end up being mainly White men and women. 

I chose to select only the documents containing information from 2006-2012 and 2013-2020. These sheets were loosely formatted with a larger section ("Food") contianing some amount of rows beneath it with more specific items like "Meat", which could also contain subsets like "Pork". 
<div align="center">
  <img
    src="../assets/img/sampleraw.png"
    width="60%"
    style="border: 2px solid"
    alt="Raw sample of expenditure sheet"
  />
  <figcaption style="font-style: italic">
    Unrefined dataset.
  </figcaption>
</div>

I decided that for my purposes I was not interested in the particulars of the spending, just the bigger picture. Because of this my first step was to clean the datasets to contain only the most important information. To do this, I first manually edited the sheets to remove non-monetary data. This included things like "Average number in consumer unit" (meaning household size) or the percent distribution of various demographic information of the reference individual. This information may be interesting but it will not serve a dataset that we want to manipulate in Python. After doing this, I took my version of the dataset and began working with Claude AI to further clean and organize the data. I instructed it to look for the Items at the beginning of a row break, as each main section is separated this way, and only include that first term.


In [5]:
# Function to process the data and group by categories
def process_consumer_spending(df):
    # Forward fill NaN values in first column to identify categories
    df.iloc[:, 0] = df.iloc[:, 0].fillna('')
    
    # Initialize variables
    categories = []
    current_category = None
    grouped_data = []
    
    for index, row in df.iterrows():
        item = row.iloc[0]
        
        # Skip completely empty rows
        if item == '':
            current_category = None
            continue
        
        # Check if this is a main category (no indentation and not a subcategory)
        if not item.startswith(' ') and ',' not in item and ':' not in item:
            # We found a new main category
            current_category = item
            if current_category not in categories:
                categories.append(current_category)
        
        # Create a row with category information
        new_row = row.copy()
        if current_category:
            new_row['Category'] = current_category
            grouped_data.append(new_row)
    
    # Convert to DataFrame
    result_df = pd.DataFrame(grouped_data)
    
    # Clean up the monetary values
    for year in range(2006, 2013):
        year_col = str(year)
        if year_col in result_df.columns:
            result_df[year_col] = result_df[year_col].apply(lambda x: 
                pd.to_numeric(str(x).replace('$', '').replace(',', ''), errors='coerce')
                if isinstance(x, str) else x)
    
    # Rename columns for clarity
    result_df.rename(columns={result_df.columns[0]: 'Item'}, inplace=True)
    
    return result_df

This worked really well and I was left with the categories:

1	Average Annual Expenditures

2	Food

3	Alcoholic beverages

4	Housing

5	Apparel and services

6	Transportation

7	Healthcare

8	Entertainment

9	Personal care products and services

10	Reading

11	Education

12	Tobacco products and smoking supplies

13	Miscellaneous

14	Cash contributions

15	Personal insurance and pensions

16	Sources of income and personal taxes:

17	Money income before taxes a/

18	Personal taxes (missing values not imputed) a/

19	Income after taxes a/

20	Addenda:

21	Net change in total assets and liabilities

22	Other financial information:

23	Income before taxes

24	Personal taxes (contains some imputed values)

25	Income after taxes

A few of these items have no data associated with them, but as far as I could tell it did not affect the computation of the results, so I decided I was okay with keeping some blank rows. 

After doing this for both sets of data and double checking to make sure all the categories lined up, I used Python to merge the two cleaned csv files.  

In [9]:
# Function to read and clean the CSV files
def read_and_clean_csv(filename):
    # Read the CSV file
    df = pd.read_csv(filename)
    
    # Clean column names by stripping quotes and whitespace
    df.columns = df.columns.str.strip('"').str.strip()
    
    # For the first column, remove quotation marks and whitespace
    df.iloc[:, 0] = df.iloc[:, 0].str.strip('"').str.strip() if df.iloc[:, 0].dtype == 'object' else df.iloc[:, 0]
    
    return df

# Function to extract main categories
def extract_main_categories(df):
    # Identify main categories by finding rows that come right after blank rows
    # First, identify blank rows (rows where the first column is empty or NaN)
    blank_rows = df.iloc[:, 0].isna() | (df.iloc[:, 0] == '')
    
    # Find rows that come right after blank rows
    main_category_indices = []
    for i in range(1, len(blank_rows)):
        if blank_rows[i-1] and not blank_rows[i]:
            main_category_indices.append(i)
    
    # Also include the first row (which contains "Average Annual Expenditure")
    if not blank_rows[0]:
        main_category_indices.insert(0, 0)
    
    # Extract main categories and their data
    main_categories_df = df.iloc[main_category_indices].copy()
    
    return main_categories_df

# Read and clean the CSV files
df1 = read_and_clean_csv('../data/original/consumer06csv.csv')
df2 = read_and_clean_csv('../data/original/consumer13csv.csv')

# Extract main categories from each file
main_categories_df1 = extract_main_categories(df1)
main_categories_df2 = extract_main_categories(df2)

# Convert dollar values to numeric, removing $ and commas
def clean_dollar_values(df):
    for col in df.columns[1:]:  # Skip the first column (Item)
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace('$', '', regex=False)
            df[col] = df[col].str.replace(',', '', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

main_categories_df1 = clean_dollar_values(main_categories_df1)
main_categories_df2 = clean_dollar_values(main_categories_df2)

# Combine the two dataframes
# First, prepare the dataframes by setting the 'Item' column as index
main_categories_df1.set_index('Item', inplace=True)
main_categories_df2.set_index('Item', inplace=True)

# Now combine them
combined_df = pd.concat([main_categories_df1, main_categories_df2], axis=1)

# Reset index to make 'Item' a column again
combined_df.reset_index(inplace=True)

# Display the combined dataframe
print("Combined data for main spending categories (2006-2020):")
print(combined_df)

# Save the combined data to a new CSV file
combined_df.to_csv('combined_consumer_spending_2006_2020.csv', index=False)
print("\nData saved to 'combined_consumer_spending_2006_2020.csv'")

NameError: name 'pd' is not defined

Next, in order to make the dataframe a bit more readable for Python, I used the command '.long' to transform the dataframe into a "long" format as opposed to a "wide" one; meaning each item for each year is given its own row, as oppposed to four columns of years by the amount of items. Somehow this also removed the rows with no value, which was a wonderful bonus. Then I began exploring the data and looking for trends, as well as using Claude to help me examine individual items or specific areas.

In [12]:
# Read the CSV file
df = pd.read_csv('combined_consumer_spending_2006_2020.csv')

# Clean up the data first by removing rows that contain summaries or non-spending categories
# We'll keep only the main spending categories
main_categories = [
    'Average Annual Expenditures', 'Food', 'Alcoholic beverages', 'Housing', 
    'Apparel and services', 'Transportation', 'Healthcare', 'Entertainment',
    'Personal care products and services', 'Reading', 'Education', 
    'Tobacco products and smoking supplies', 'Miscellaneous',
    'Cash contributions', 'Personal insurance and pensions'
]

# Filter the dataframe to keep only the main spending categories
df_clean = df[df['Item'].isin(main_categories)]

# Now melt the dataframe to convert from wide to long format
df_long = pd.melt(
    df_clean,
    id_vars=['Item'],  # Keep 'Item' as an identifier
    value_vars=[str(year) for year in range(2006, 2021)],  # Years 2006-2020 as variables
    var_name='Year',  # Name for the new column containing years
    value_name='Spending'  # Name for the new column containing spending values
)

# Convert Year to integer and Spending to float
df_long['Year'] = df_long['Year'].astype(int)
df_long['Spending'] = pd.to_numeric(df_long['Spending'], errors='coerce')

# Display the first few rows of the transformed data
print(df_long.head(10))

# Save the transformed data to a new CSV file
df_long.to_csv('consumer_spending_long_format.csv', index=False)

NameError: name 'pd' is not defined

### Main Analysis
To begin with, there is some interesting information not contained in the graphs I created. Luckily the kind of information it is does not require in-depth processing to understand. Over all 14 years, the biggest gap between the male-female ration was 46:54 in 2006. In the following years it stayed at 47:53 into and through the 2010s with a few 48:52 seemingly at random. The distribution by race is much less balanced; almost all of the years on record here the White population representing around 87% of the reference group, with black or latino only being about 13%. 

Another extremely relevant statistic recorded in these documents was the percentage of homeowners surveyed who had a mortgage. The housing crisis of 2007-2008 was due to the "bursting" of the "housing bubble", essentially meaning that housing prices rose steeply and quickly which led to many more people missing payments, which in turn hurt the housing market and caused prices to plummet. Because of this, there was supposedly less desire or need for mortages. Indeed, we can see this reflected in a chart of the percentages of homeowners and renters:
<div align="center">
  <img
    src="../assets/img/homeowners_trends.png"
    width="60%"
    style="border: 2px solid"
    alt="Homeowner Trends"
  />
  <figcaption style="font-style: italic">
    Percent of Homeowners with mortage, without mortage, or renting by Year
  </figcaption>
</div>
Not only does the amount of homeowners paying a mortage go down, but renters actually overtake the homeowners with mortgages in 2015. Another really interesting bit of information is that the percent of people who had completed college rose steadily from 59% in 2006 to 69% in 2020.

As for the actual expenditures, I made a few broad charts showing the items with the most budget share and the percent increase or decrease each year.
<div align="center">
  <img
    src="../assets/img/categories-budgetshare.png"
    width="60%"
    style="border: 2px solid"
    alt="Top categories"
  />
  <figcaption style="font-style: italic">
    The categories with the largest budget share.
  </figcaption>
</div>

This shows very clearly that, although the housing market took a hit after 2008, housing prices quickly began to climb again, and now we are spending more than ever before on housing. The other four categories Transportation, Apparel, Insurance, and Healthcare also show a gradual rise over the years, aside from minor dips in 2009-2010 and 2019-2020 in the Transportation and Apparel categories. 

<div align="center">
  <img
    src="../assets/img/percent-change.png"
    width="60%"
    style="border: 2px solid"
    alt="% Change in budget share"
  />
  <figcaption style="font-style: italic">
    The change in the percent of total budget share for top 5 categories.
  </figcaption>
</div>

This shows the actual percent change by year for the main 6 categories. With this chart it is easier to see that people began spending about 18% less on alcohol following the Great Recession, while they actually spent about 18% *more* on tobacco. It is also interesting to note the rapid increase in cost of education.

Here are the two highest value categories comapred to each other:
<div align="center">
  <img
    src="../assets/img/high-value.png"
    width="60%"
    style="border: 2px solid"
    alt="Main Spendingh"
  />
  <figcaption style="font-style: italic">
    Housing and food expenditures
  </figcaption>
</div>

And the lower value categories:
<div align="center">
  <img
    src="../assets/img/low-value.png"
    width="60%"
    style="border: 2px solid"
    alt="Other Spending"
  />
  <figcaption style="font-style: italic">
    Other Spending.
  </figcaption>
</div>

The BLS also has data on employment information from this same time period. Looking at the columns for the change between 2007 and 2010, the Federal Government was the only sector to increase in employment, while all others lost between 1% and 28% of the workforce. 
<div align="center">
  <img
    src="../assets/img/blsemployment07.png"
    width="60%"
    style="border: 2px solid"
    alt="BLS Employment Statistics"
  />
  <figcaption style="font-style: italic">
    Full chart available from the BLS website.
  </figcaption>
</div>

Another source that is worth looking at is the data on Savings from the St. Louis government. 
<div align="center">
  <img
    src="../assets/img/savings.png"
    width="60%"
    style="border: 2px solid"
    alt="Savings"
  />
  <figcaption style="font-style: italic">
    Average savings of St. Louis residents.
  </figcaption>
</div>

With this data we can understand a little more about the economic rebound in the later 2010s. It seems clear that as the workforce began to grow again from 2012-2020, people were able to put more of their money into savings. Looking back at our Percent Change graph, we also see that people have started spending less generally on everything except housing. 

### Conclusion
Housing in America has been a persistently prominent issue for decades. Allowing too many people to apply for mortgage loans and buy property apparently created a significant amount of panic in the housing market, and they responded predatorily by jacking prices back up. Because of this, more people than ever before are renting without any sort of mortgage, and we are generally spending less on non-essential things like entertainment and alcohol. Additionally we see a trend of more money being diverted to savings, but with less mortgages and general spending, there is a question of what exactly the money is being saved for. 

There are some limitations to this data set. As mentioned in the main analysis, there is data included on demographics, but the range is very limited. Besides white, the only other options given were black or latino. Obviously this excludes a huge amount of other ethnicities and races, and in the end white people made up more than 80% of the reference group. There is also no information given regarding the regions surveyed. We do not know how much of this is representative of urban or suburban or even rural life. It is very possible that the number of people renting vs mortgaging looks extremely different between, say, New York City and a small town in Iowa. 

Still, there are obviously some trends in the data that speak for themselves. The average annual expenditures rose by about $2000 every year since 2010, and housing and education are the only categories whose share has not decreased. In fact, their budget share also seems to increase along with the total expenditures. This means that people have less and less money to spend on quality groceries, entertainment, and leisure activities. The desperate scramble for housing in the U.S is part of the reason we are so money-hungry; each year it costs more to have a roof over your head, causing great anxiety for multiple generations of Americans. Keeping money in savings so you aren't in danger of losing your home is a priority for almost everyone. More and more people are living with more and more roommates to try to keep their indivudal spending down. With the popularity of apps like Air B&B, the housing market is only growing more competitive. People are buying property with the sole intent of renting it out to someone else. This will keep driving up the demand for living spaces which will keep growing the cost, and so on and so forth. I believe that the housing bubble burst is a main factor driving this change.

## Repository and Bibliography

### Github Repository
__[Here](https://github.com/bwarzel/research-project)__

### Bibliography
Original Main Data Set: https://www.bls.gov/opub/mlr/2014/article/consumer-spending-and-us-employment-from-the-recession-through-2022.htm
Employment Statistics: https://www.bls.gov/opub/mlr/2014/article/consumer-spending-and-us-employment-from-the-recession-through-2022.htm
St. Louis Savings: https://fred.stlouisfed.org/series/W398RC1A027NBEA
2008 Consumer Spending Internal Report https://www.imf.org/external/pubs/ft/spn/2010/spn1001.pdf
